In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input, Conv1D, MaxPooling1D, Flatten
from keras.models import Model, Sequential  

df_train = pd.read_csv("./drive/My Drive/sales_train_validation.csv")
df_sell = pd.read_csv("./drive/My Drive/sell_prices.csv")
df_calendar = pd.read_csv("./drive/My Drive/calendar.csv")
df_ss = pd.read_csv("./drive/My Drive/sample_submission_m5.csv")


In [ ]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]


In [ ]:
time_series_data = df_train[time_series_cols] 
time_series_data


In [ ]:
time_series_data = np.asarray(time_series_data).astype('float32') 
print(time_series_data.shape)

In [ ]:
def build_model(timesteps, features): 
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, features)))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(features))
    return model  

In [ ]:
def split_xy2(dataset, time_steps):
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+time_steps]) 
    return np.array(x), np.array(y) 

In [ ]:
predictions = [] 

for i in range(1,29): 
    print("day {}".format(i)) 
    data = time_series_data[:,::i]
    x_train, y_train = split_xy2(data,7) 
    x_pred = data[:,data.shape[1]-7:] 
    x_train = x_train.reshape(x_train.shape[0],7,30490)  
    x_pred = x_pred.reshape(1,7,30490)
    print(x_train.shape) 
    print(x_pred.shape)  
    model = build_model(7,30490)    
    model.compile(optimizer='adam',loss='mse')  
    model.fit(x_train,y_train,epochs=150)   
    pred = model.predict(x_pred) 
    print(pred) 
    predictions.append(pred) 
